In [14]:
import cv2
import pygame
import numpy as np

In [15]:
tracker_file_path = "data/kaggle_test/test_2/test_2_tracking_data.txt"

src_video_path = 'data/kaggle_test/test_2/test_2_src.mp4'
control_area_video_path = 'data/kaggle_test/test_2/test_2_control_area.mp4'
trajectory_video_path = 'data/kaggle_test/test_2/test_2_trajectory.mp4'
target_tracker_video_path='data/kaggle_test/test_2/test_2_tracker.mp4'
target_playarea_video_path = "data/kaggle_test/test_2/test_2_pitch.mp4"
target_projection_video_path = "data/kaggle_test/test_2/test_2_final.mp4"


In [16]:
pygame.init()

# Video Paths

video_paths = [src_video_path,target_tracker_video_path,target_playarea_video_path,target_projection_video_path,control_area_video_path,trajectory_video_path]
names = ["Original", "Tracker", "Playarea", "2D Proj", "Control Area", "Trajectory"]

# Load videos using OpenCV
video_captures = [cv2.VideoCapture(path) for path in video_paths]

# Screen settings
WIDTH, HEIGHT = 960, 540
screen = pygame.display.set_mode((WIDTH, HEIGHT + 100))  # Extra space for controls
pygame.display.set_caption("Football Match Analyzer")

# Button settings
BUTTON_WIDTH, BUTTON_HEIGHT = 120, 40
button_y = 10
buttons = [pygame.Rect(10 + (BUTTON_WIDTH + 10) * i, button_y, BUTTON_WIDTH, BUTTON_HEIGHT) for i in range(len(names))]

# Font settings
font = pygame.font.Font(None, 28)

# Video controls
paused = False
active_video = 0
frame_counter = 0  # Global frame counter

running = True
clock = pygame.time.Clock()

# Get FPS from the first video
fps = int(video_captures[0].get(cv2.CAP_PROP_FPS))
total_frames = int(video_captures[0].get(cv2.CAP_PROP_FRAME_COUNT))

while running:
    screen.fill((0, 0, 0))

    # Draw buttons at the top
    for i, button in enumerate(buttons):
        color = (0, 255, 0) if i == active_video else (100, 100, 100)
        pygame.draw.rect(screen, color, button)
        text_surface = font.render(names[i], True, (255, 255, 255))
        screen.blit(text_surface, button.move(10, 10))

    # Read frames only if not paused
    if not paused:
        frame_counter += 1
        if frame_counter >= total_frames:
            frame_counter = 0  # Restart all videos
            for cap in video_captures:
                cap.set(cv2.CAP_PROP_POS_FRAMES, 0)

        # Read frames from all videos
        frames = []
        for cap in video_captures:
            ret, frame = cap.read()
            if not ret:  # Restart if video ends
                cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
                ret, frame = cap.read()
            frames.append(frame)

    # Display the selected video frame
    frame = frames[active_video] if frames else None
    if frame is not None:
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame = cv2.flip(frame, 1)
        frame = cv2.resize(frame, (WIDTH, HEIGHT))
        frame = np.rot90(frame)
        frame = pygame.surfarray.make_surface(frame)
        screen.blit(frame, (0, 70))

    # Video progress bar
    progress = frame_counter / total_frames if total_frames > 0 else 0
    progress_bar = pygame.Rect(10, HEIGHT + 60, int((WIDTH - 20) * progress), 10)
    pygame.draw.rect(screen, (200, 0, 0), progress_bar)

    # Pause button
    pause_button = pygame.Rect(WIDTH // 2 - 40, HEIGHT + 30, 80, 30)
    pygame.draw.rect(screen, (200, 200, 200), pause_button)
    pause_text = font.render("Pause" if not paused else "Play", True, (0, 0, 0))
    screen.blit(pause_text, pause_button.move(10, 10))

    # Event handling
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        elif event.type == pygame.MOUSEBUTTONDOWN:
            for i, button in enumerate(buttons):
                if button.collidepoint(event.pos):
                    active_video = i  # Switch active video
            if pause_button.collidepoint(event.pos):
                paused = not paused
            elif 10 <= event.pos[0] <= WIDTH - 10 and HEIGHT + 60 <= event.pos[1] <= HEIGHT + 70:
                # Seek all videos
                frame_counter = int((event.pos[0] - 10) / (WIDTH - 20) * total_frames)
                for cap in video_captures:
                    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_counter)

    pygame.display.update()
    clock.tick(fps)  # Maintain consistent playback speed

pygame.quit()
for cap in video_captures:
    cap.release()
cv2.destroyAllWindows()
